In [30]:
from sklearn.model_selection import train_test_split
import pickle 
import os
from pathlib import Path
import pandas as pd
from datasets import Dataset

In [15]:
curr_dir = Path(os.getcwd()).parent
with open(f"{curr_dir}/Data/train_processed", "rb") as f:
    dftr = pickle.load(f)

In [18]:
train_df, eval_df = train_test_split(dftr, test_size=0.2, stratify = dftr["label"], random_state=42)

In [22]:
# Load model directly
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [59]:
# import bert model and introduce random dropout and label classifier
from transformers import BertModel
import torch.nn as nn

class CustomBERTModel(nn.Module):
    def __init__(self):
        super(CustomBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained("google-bert/bert-base-uncased")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 3)
        self.bert.gradient_checkpointing_enable()

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask) # bert output
        output = self.dropout(output.pooler_output) # dropout
        logits = self.classifier(output) # classifier

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1)) # loss calculation

        return loss, logits

model = CustomBERTModel()

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: google-bert/bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [60]:
# tokenize the data
def tokenize(batch):
    tokens = tokenizer(batch['text'], truncation=True,
                       padding = True)
    return tokens

In [62]:
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

In [63]:
# map the tokenize function to the dataset
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
eval_dataset = eval_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))

Map:   0%|          | 0/16845 [00:00<?, ? examples/s]

Map:   0%|          | 0/4212 [00:00<?, ? examples/s]

In [64]:
# set the format to pytorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [68]:
# set the training arguments
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4.1, # total number of training epochs
    per_device_train_batch_size=32, # batch size per device during training
    per_device_eval_batch_size=32, # batch size for evaluation
    warmup_steps=100, # number of warmup steps for learning rate scheduler
    weight_decay=0.01, # strength of weight decay
    learning_rate = 1.75e-5, # learning rate
    logging_dir=f"{curr_dir}/Output/logs",
    logging_steps=10, # logging steps
    metric_for_best_model='accuracy',  # Use accuracy for early stopping evaluation
    fp16=True
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [69]:
# define the trainer
trainer = Trainer(
    model=model, # the instantiated hf Transformers model to be trained
    args= training_args, # training arguments, defined above
    train_dataset=train_dataset, # training dataset
    eval_dataset=eval_dataset # evaluation dataset
)

In [ ]:
trainer.train()

/Users/chengsun/miniconda3/envs/beer_review_nlp/lib/python3.10/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  super().__init__(loader)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.148488
20,1.140639
30,1.122733
40,1.160925
50,1.162560
60,1.140385
70,1.157173
80,1.157649
90,1.138133
100,1.166641


Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG14GFamilyCommandBuffer: 0x386635880>
    label = <none> 
    device = <AGXG14GDevice: 0x3c7026200>
        name = Apple M2 
    commandQueue = <AGXG14GFamilyCommandQueue: 0x3c780b400>
        label = <none> 
        device = <AGXG14GDevice: 0x3c7026200>
            name = Apple M2 
    retainedReferences = 1
Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG14GFamilyCommandBuffer: 0x33f3911c0>
    label = <none> 
    device = <AGXG14GDevice: 0x3c7026200>
        name = Apple M2 
    commandQueue = <AGXG14GFamilyCommandQueue: 0x3c780b400>
        label = <none> 
        device = <AGXG14GDev